In [6]:
from matplotlib import pyplot as plt
from systemclass import SNAIL,SNAIL_sweep,Cavity, SNAILC, SNAILC_sweep, SNAILCC, SNAILCC_sweep
import numpy as np
from joblib import Parallel, delayed
import qutip as qt
from qutip import propagator,floquet_modes,Qobj,Options,basis
from scipy.optimize import fsolve
import numpy as np
def state_index(index,dim):
    m,n,k = index
    M,N,K = dim
    return m*(N*K)+n*K+k
def sort_eigenpairs(eigenvalues, eigenvectors):
    n = eigenvectors.shape[0]
    sorted_indices = []

    for i in range(n):
        max_abs_vals = np.abs(eigenvectors[i, :])
        max_index = np.argmax(max_abs_vals)
        while max_index in sorted_indices:
            max_abs_vals[max_index] = -np.inf
            max_index = np.argmax(max_abs_vals)
        sorted_indices.append(max_index)

    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]

    return sorted_eigenvalues, sorted_eigenvectors
    
    return sorted_eigenvalues, sorted_eigenvectors
def find_max_amplitude(duration, target_integral=6700):
    # Standard deviation of the Gaussian pulse
    sigma = duration  / 3
    center = duration /2
    # Define the equation to solve: integral of the Gaussian minus the target integral
    def equation(amplitude):
        # Integral of a Gaussian from -inf to inf is sqrt(2*pi) * sigma * amplitude
        # Here, we approximate that the integral from 0 to duration is a good representation
        return amplitude * np.sqrt(2 * np.pi) * sigma - target_integral -amplitude * np.exp(-0.5 * (- center / sigma) ** 2) *duration 

    # Initial guess for amplitude
    initial_guess = 1

    # Solve for amplitude
    amplitude_solution = fsolve(equation, initial_guess)

    return amplitude_solution[0]
def gaussian_pulse(t0,duration,t1, max_amplitude):
    # Define the time array centered at duration/2
    new_duration = t1 - t0
    # Standard deviation for the Gaussian pulse
    sigma = new_duration / 3 # Assuming the pulse mostly lies within +/- 3 sigma
    
    # Shift the center of the pulse to duration/2
    center = new_duration/ 2
    
    # Generate the Gaussian pulse
    def time_dependent_function(t):
        if t <= t0:
            pulse = 0
        elif t <= t1:
            t = t-t0
            pulse =  max_amplitude * np.exp(-0.5 * ((t - center) / sigma) ** 2)-max_amplitude * np.exp(-0.5 * (- center / sigma) ** 2)
        else:
            pulse =  0
        return pulse
    return time_dependent_function
def gaussian_ramp(t0,t1, amplitude):
    """
    Creates a time-dependent function using a Gaussian function that starts near zero,
    peaks at 'amplitude' at time 't0', and remains constant at 'amplitude' for t > t0.

    Args:
    t0 (float): The time at which the function peaks at the amplitude.
    amplitude (float): The maximum amplitude of the Gaussian peak.

    Returns:
    function: A function of time 't' that implements the desired behavior.
    """
    # Define the standard deviation such that the peak is sharp at t0
    sigma = t0 / 3  # Adjust sigma to make the function start close to zero at t=0
    
    offset = amplitude * np.exp(-((0 - t0)**2) / (2 * sigma**2))
    
    # Define the function using a closure to encapsulate the parameters
    def time_dependent_function(t):
        if t <= t0:
            # Gaussian function for ramp up
            return amplitude * np.exp(-((t - t0)**2) / (2 * sigma**2)) - offset
        elif t <= t1:
            # Constant amplitude between t0 and t1
            return amplitude - offset
        else:
            # Symmetric Gaussian function for ramp down
            return amplitude * np.exp(-((t - t1)**2) / (2 * sigma**2)) - offset

    return time_dependent_function

def energy_diff(t0,duration,t1,initial_duration,omega):
    flux =0.432
    #snail parameters
    EJ = 100*2*np.pi
    EC = 0.177*2*np.pi
    beta = 0.12
    ng = 0
    ncut = 300
    N = 3
    dims = 18
    snail = SNAIL(EJ,EC,beta,ng,ncut,flux,N,dims)
    Hs,charge_op = snail.spectrum_charge()


    #cavity1 parameters
    omega_c1 = 4.5*2*np.pi
    g_int1 = 0.15/2*(6-omega_c1/2/np.pi)*2*np.pi
    dimc1 = 4
    cavity1 = Cavity(omega_c1,g_int1,dimc1)

    #cavity1 parameters
    omega_c2 =  8.5*2*np.pi
    g_int2 = 0.15/2*(6-omega_c2/2/np.pi)*2*np.pi
    dimc2 = 4
    cavity2 = Cavity(omega_c2,g_int2,dimc2)

    Hc1, Vc1 = cavity1.hamiltonian()
    Hc2, Vc2 = cavity2.hamiltonian()

    Ic1 = np.identity(dimc1)
    Ic2 = np.identity(dimc2)
    Is = np.identity(dims)

    Hs = np.kron(np.kron(Hs,Ic1),Ic2)
    Hc1 = np.kron(np.kron(Is,Hc1),Ic2)
    Hc2 = np.kron(np.kron(Is,Ic1),Hc2)

    H_int1 = cavity1.g_int * np.kron(np.kron(charge_op,Vc1),Ic2) * 2 * ((2 * snail.EC / snail.EJ) ** 0.25)
    H_int2 = cavity2.g_int * np.kron(np.kron(charge_op,Ic1),Vc2) * 2 * ((2 * snail.EC / snail.EJ) ** 0.25)

    H = Hs + Hc1 + Hc2 + H_int1 + H_int2
    Hc  =  np.kron(np.kron(charge_op,Ic1),Ic2)
    energy0,U = np.linalg.eigh(H)
    energy0,U = sort_eigenpairs(energy0, U)
    Ud = U.transpose().conjugate()
    H = Ud@H@U
    Hc = Ud@Hc@U

    index1 = np.argmin(np.abs(energy0 - omega_c1 * np.ones(len(energy0))))
    index2 = np.argmin(np.abs(energy0 - omega_c2 * np.ones(len(energy0))))

    total_dim = dims*dimc1*dimc2
    H0 = Qobj(H)
    Hc = Qobj(Hc)
    psi0 = qt.basis(total_dim, index1)
    Iss = qt.qeye(dims)
    s0 = qt.basis(dims, 0)*(qt.basis(dims, 0).dag())
    cavity0 = qt.basis(dimc1, 0)*(qt.basis(dimc1, 0).dag())
    cavity1 = qt.basis(dimc1, 1)*(qt.basis(dimc1, 1).dag())
    cavity2 = qt.basis(dimc1, 2)*(qt.basis(dimc1, 2).dag())
    PP1 = qt.Qobj(np.array(qt.tensor(Iss, cavity0, cavity1)))
    PP2 = qt.Qobj(np.array(qt.tensor(Iss, cavity1, cavity0)))
    omega1 =  5.8* 2 * np.pi
#     initial_duration = 2000
    amplitude = find_max_amplitude(initial_duration,  target_integral=6700)
    func = gaussian_pulse(t0,duration,t1, amplitude )
    func1 = gaussian_ramp(t0 ,t1,0.003 * 2 * np.pi)
#     omega = 4.0104625*2*np.pi
    
    args = {'w': omega,'w1':omega1}
    tlist = np.linspace(0, duration, 100000)  # Cover ten periods
    H = [H0, [Hc, lambda t, args: func(t)*np.cos(args['w']*t)],[Hc, lambda t, args: func1(t)*np.cos(args['w1']*t)]]

    result = qt.sesolve(H, psi0, tlist, args=args)
    return result

In [9]:
t0 = 200
t1s = np.linspace(t0+660, t0+680, 10)  # 11 points for 10 intervals
omega_values = np.linspace(4.01037,4.0104,5) * 2 * np.pi
# Define the range for omega values, scaling by 2*pi
initial_duration = 500
t1 = 968.9
# Use joblib to parallelize the calculation over both durations and omega values
results = (energy_diff)(t0,t1+0, t1,initial_duration, 4.010325*2*np.pi)